In [1]:
import os
import promptflow

root = os.path.join(os.getcwd(), "../")
flow_path = os.path.join(root, "named-entity-recognition")
data_path = os.path.join(flow_path, "data.jsonl")
eval_match_rate_flow_path = os.path.join(root, "../evaluation/eval-entity-match-rate")

pf = promptflow.PFClient()


In [2]:
# Run the flow
run = pf.run(
    flow=flow_path,
    data=data_path,
    column_mapping={
        "text": "${data.text}",
        "entity_type": "${data.entity_type}"
    },
    display_name="ner_bulk_run",
    tags={"unittest": "true"},
    stream=True)

2023-08-28 14:34:26 +0800   38388 execution          INFO     Start to run 2 nodes with concurrency level 2.
2023-08-28 14:34:26 +0800   38388 execution.flow     INFO     Executing node NER_LLM. node run id: named_entity_recognition_default_20230828_143413_282033_NER_LLM_1
2023-08-28 14:34:26 +0800   41368 execution          INFO     Start to run 2 nodes with concurrency level 2.
2023-08-28 14:34:26 +0800   41368 execution.flow     INFO     Executing node NER_LLM. node run id: named_entity_recognition_default_20230828_143413_282033_NER_LLM_12
2023-08-28 14:34:26 +0800   14748 execution          INFO     Start to run 2 nodes with concurrency level 2.
2023-08-28 14:34:26 +0800   14748 execution.flow     INFO     Executing node NER_LLM. node run id: named_entity_recognition_default_20230828_143413_282033_NER_LLM_6
2023-08-28 14:34:26 +0800   27332 execution          INFO     Start to run 2 nodes with concurrency level 2.
2023-08-28 14:34:26 +0800   27332 execution.flow     INFO     Execut

In [3]:
pf.get_details(run)

,inputs.entity_type,inputs.text,inputs.line_number,outputs.entities
0,job title,The software engineer is working on a new upda...,0,[software engineer]
1,job title,The project manager and the data analyst are c...,1,"[project manager, data analyst]"
2,job title,The marketing manager is coordinating with the...,2,"[marketing manager, graphic designer]"
3,job title,The CEO and CFO are discussing the financial f...,3,"[CEO, CFO]"
4,job title,The web developer and UX designer are working ...,4,"[web developer, UX designer]"
5,phone number,John finally decided to change his phone numbe...,5,[None]
6,phone number,"If you have any questions about our products, ...",6,[(123) 456-7890]
7,phone number,"My new phone number is (098) 765-4321, please ...",7,[(098) 765-4321]
8,phone number,The phone number (321) 654-0987 is no longer i...,8,[(321) 654-0987]
9,phone number,Please dial the following phone number: (555) ...,9,[(555) 123-4567]


In [4]:
eval = pf.run(
    flow=eval_match_rate_flow_path,
    run=run,
    data=data_path,
    column_mapping={
        "entities": "${run.outputs.entities}",
        "ground_truth": "${data.results}"
    },
    display_name="eval_match_rate",
    tags={"unittest": "true"},
    stream=True)

pf.get_details(eval)

2023-08-28 14:34:42 +0800   28348 execution          INFO     Start to run 2 nodes with concurrency level 2.
2023-08-28 14:34:42 +0800   28348 execution.flow     INFO     Executing node cleansing. node run id: eval_entity_match_rate_default_20230828_143433_845944_cleansing_7
2023-08-28 14:34:42 +0800   28348 execution.flow     INFO     Node cleansing completes.
2023-08-28 14:34:42 +0800   28348 execution.flow     INFO     Executing node match. node run id: eval_entity_match_rate_default_20230828_143433_845944_match_7
2023-08-28 14:34:42 +0800   28348 execution.flow     INFO     Node match completes.
2023-08-28 14:34:43 +0800   51660 execution          INFO     Start to run 2 nodes with concurrency level 2.
2023-08-28 14:34:43 +0800   51660 execution.flow     INFO     Executing node cleansing. node run id: eval_entity_match_rate_default_20230828_143433_845944_cleansing_10
2023-08-28 14:34:43 +0800    4836 execution          INFO     Start to run 2 nodes with concurrency level 2.
2023-08

,inputs.entities,inputs.ground_truth,inputs.line_number,outputs.match_cnt
0,[software engineer],software engineer,0,"{'exact_match': 1, 'partial_match': 1, 'answer..."
1,"[project manager, data analyst]","project manager, data analyst",1,"{'exact_match': 1, 'partial_match': 1, 'answer..."
2,"[marketing manager, graphic designer]","marketing manager, graphic designer",2,"{'exact_match': 1, 'partial_match': 1, 'answer..."
3,"[CEO, CFO]","CEO, CFO",3,"{'exact_match': 1, 'partial_match': 1, 'answer..."
4,"[web developer, UX designer]","web developer, UX designer",4,"{'exact_match': 1, 'partial_match': 1, 'answer..."
5,[None],None,5,"{'exact_match': 1, 'partial_match': 1, 'answer..."
6,[(123) 456-7890],(123) 456-7890,6,"{'exact_match': 1, 'partial_match': 1, 'answer..."
7,[(098) 765-4321],(098) 765-4321,7,"{'exact_match': 1, 'partial_match': 1, 'answer..."
8,[(321) 654-0987],(321) 654-0987,8,"{'exact_match': 1, 'partial_match': 1, 'answer..."
9,[(555) 123-4567],(555) 123-4567,9,"{'exact_match': 1, 'partial_match': 1, 'answer..."


In [5]:
pf.get_metrics(eval)

{'exact_match_rate': 1.0, 'partial_match_rate': 1.0}

In [6]:
pf.visualize([run, eval])

The HTML file is generated at 'C:\\Users\\lisal\\AppData\\Local\\Temp\\pf-visualize-detail-2lvut0rc.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
